In [1]:
import concurrent.futures
from pathlib import Path

from agent import Agent
from constants import CONSISTENT_TEMPERATURE, CREATIVE_TEMPERATURE
from prompts import (
    PRINCIPAL_INVESTIGATOR,
    SCIENTIFIC_CRITIC,
    create_merge_prompt,
)
from run_individual_meeting import run_individual_meeting
from run_scientific_meeting import run_scientific_meeting
from utils import load_summaries

In [2]:
# Set up key parameters
num_iterations = 5
num_rounds = 3
num_critiques = 3
save_dir = Path("antibody_design")
model = "gpt-4o"
background_prompt = "You are working on a research project to use machine learning to develop antibodies for the newest variant of the SARS-CoV-2 spike protein that also, ideally, have activity against other circulating minor variants and past variants."

## Select team members

In [3]:
# Select team members - prompts
team_selection_dir = save_dir / "team_selection"

team_selection_agenda = f"""{background_prompt} You need to select a team of three scientists to help you with this project. Please select the team members that you would like to invite to a discussion to create the antibody design approach. Please list the team members in the following format, using the team member below as an example. You should not include yourself (Principal Investigator) in the list.

Agent(
    title="Principal Investigator",
    expertise="applying artificial intelligence to biomedical research",
    goal="perform research in your area of expertise that maximizes the scientific impact of the work",
    role="lead a team of experts to solve an important problem in artificial intelligence for biomedicine, make key decisions about the project direction based on team member input, and manage the project timeline and resources",
)
"""

In [ ]:
# Select team members - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=PRINCIPAL_INVESTIGATOR,
            agenda=team_selection_agenda,
            save_dir=team_selection_dir,
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

In [ ]:
# Select team members - merge
team_selection_summaries = load_summaries(discussion_paths=list(team_selection_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(team_selection_summaries)}")

team_selection_merge_prompt = create_merge_prompt(agenda=team_selection_agenda)

run_individual_meeting(
    team_member=PRINCIPAL_INVESTIGATOR,
    agenda=team_selection_merge_prompt,
    save_dir=team_selection_dir,
    save_name="merged",
    summaries=team_selection_summaries,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

In [6]:
# Add team members
IMMUNOLOGIST = Agent(
    title="Immunologist",
    expertise="antibody development, immune response mechanisms, and viral pathogenesis",
    goal="identify key epitopes on the spike protein that are likely to elicit a strong immune response and assess the potential cross-reactivity with other variants",
    role="guide the selection of target epitopes for antibody design, ensuring that they are likely to be broadly neutralizing across multiple variants, and validate the effectiveness of antibody candidates",
)

COMPUTATIONAL_BIOLOGIST = Agent(
    title="Computational Biologist",
    expertise="protein structure prediction, molecular dynamics simulations, and structural bioinformatics",
    goal="model the 3D structure of the new SARS-CoV-2 spike protein variant and simulate its interactions with potential antibody candidates",
    role="provide structural insights into the spike protein, predict how mutations might affect antibody binding, and guide the computational design of antibody candidates",
)

DATA_SCIENTIST = Agent(
    title="Machine Learning Specialist",
    expertise="deep learning, predictive modeling, and biological data analysis",
    goal="develop and implement machine learning models to predict antibody efficacy and optimize antibody sequences",
    role="create and refine algorithms to predict antibody binding affinity and cross-reactivity with various SARS-CoV-2 variants, and optimize antibody sequences for broad neutralizing activity",
)

team_members = (
    IMMUNOLOGIST,
    COMPUTATIONAL_BIOLOGIST,
    DATA_SCIENTIST,
    SCIENTIFIC_CRITIC,
)

## Select project

In [4]:
# Select antibodies project - prompts
project_selection_dir = save_dir / "project_selection"

project_selection_agenda = f"{background_prompt} Please create an antibody design approach for this target that uses ESM amino acid likelihoods to identify promising antibody sequences and RosettaAntibody to rank those proposed antibodies by binding affinity. Decide whether you will design antibodies de novo or modify existing antibodies. Explain in detail how you will apply ESM and RosettaAntibody to design antibodies."

project_selection_questions = (
    "Will you design antibodies de novo or will you modify existing antibodies (choose only one)?",
    "If modifying existing antibodies, which precise antibody or antibodies will you modify?",
    "If modifying existing antibodies, how exactly will you propose modifications to the antibodies?",
    "If designing antibodies de novo, how exactly will you propose antibody candidates?",
    "How will you use ESM and RosettaAntibody (step-by-step, in detail, without any additional tools) to design antibodies?",
    "What specific role will ESM play in the antibody design process?",
    "What specific role will RosettaAntibody play in the antibody design process?",
)

project_selection_rules = (
    "You must not use any computational tools at any point in your antibody design approach except for ESM and RosettaAntibody.",
    "You must only consider the computational aspects of antibody design and ignore experimental validation.",
)

In [ ]:
# Select antibodies project - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_scientific_meeting,
            team_lead=PRINCIPAL_INVESTIGATOR,
            team_members=team_members,
            agenda=project_selection_agenda,
            agenda_questions=project_selection_questions,
            agenda_rules=project_selection_rules,
            save_dir=project_selection_dir,
            save_name=f"discussion_{iteration_num + 1}",
            num_rounds=num_rounds,
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

In [ ]:
# Select antibodies project - merge
project_selection_summaries = load_summaries(discussion_paths=list(project_selection_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(project_selection_summaries)}")

project_selection_merge_prompt = create_merge_prompt(
    agenda=project_selection_agenda,
    agenda_questions=project_selection_questions,
    agenda_rules=project_selection_rules
)

run_individual_meeting(
    team_member=PRINCIPAL_INVESTIGATOR,
    agenda=project_selection_merge_prompt,
    save_dir=project_selection_dir,
    save_name="merged",
    summaries=project_selection_summaries,
    num_critiques=num_critiques,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

## Select antibodies

In [9]:
# Select antibodies - prompts
select_antibodies_dir = save_dir / "select_antibodies"

select_antibodies_agenda = f"{background_prompt} Your team has decided to modify existing antibodies to improve their activity against the latest variant. Please describe the method you would use to determine which antibodies are most appropriate for this purpose. Then, list the specific antibodies that you would select for modification."

In [ ]:
# Select antibodies - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=IMMUNOLOGIST,
            agenda=select_antibodies_agenda,
            save_dir=select_antibodies_dir,
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
            num_critiques=num_critiques,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

In [ ]:
# Select antibodies - merge
select_antibodies_summaries = load_summaries(discussion_paths=list(select_antibodies_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(select_antibodies_summaries)}")

antibodies_merge_prompt = create_merge_prompt(agenda=select_antibodies_agenda)

run_individual_meeting(
    team_member=IMMUNOLOGIST,
    agenda=antibodies_merge_prompt,
    save_dir=select_antibodies_dir,
    save_name="merged",
    summaries=select_antibodies_summaries,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

## Get antibodies

In [ ]:
# Get antibodies - prompts
antibodies_dir = save_dir / "antibodies"

antibodies_agenda = f"{background_prompt} Your team previously decided on an antibody design approach using ESM and RosettaAntibody to modify existing antibodies. First, write out the list of existing antibodies that were selected in the previous meeting. Then, write step-by-step instructions for obtaining the sequences of each of these antibodies."

antibodies_prior_summaries = load_summaries(discussion_paths=[project_selection_dir / "merged.json"])
print(f"Number of prior summaries: {len(antibodies_prior_summaries)}")

In [ ]:
# Get antibodies - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=IMMUNOLOGIST,
            agenda=antibodies_agenda,
            save_dir=antibodies_dir,
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
            num_critiques=num_critiques,
            summaries=antibodies_prior_summaries,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

In [ ]:
# Get antibodies - merge
antibodies_summaries = load_summaries(discussion_paths=list(antibodies_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(antibodies_summaries)}")

antibodies_merge_prompt = create_merge_prompt(agenda=antibodies_agenda)

run_individual_meeting(
    team_member=IMMUNOLOGIST,
    agenda=antibodies_merge_prompt,
    save_dir=antibodies_dir,
    save_name="merged",
    summaries=antibodies_summaries,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

## Coding rules

In [7]:
coding_rules = (
    "Your code must be self-contained (with appropriate imports) and complete.",
    "Your code may not include any undefined or unimplemented variables or functions.",
    "Your code may not include any pseudocode; it must be fully functioning code.",
    "Your code may not include any hard-coded examples.",
    "If your code needs user-provided values, write code to parse those values from the command line.",
    "Your code must be high quality, well-engineered, efficient, and well-documented (including docstrings, comments, and Python type hints if using Python).",
)

## Run ESM

In [ ]:
# Run ESM - prompts
esm_dir = save_dir / "esm"

esm_agenda = f"{background_prompt} Your team previously decided on an antibody design approach using ESM and RosettaAntibody to modify existing antibodies. Now, you must implement the ESM portion of the approach. Please write a complete Python script that takes an antibody sequence as input and uses ESM amino acid likelihoods to identify the most promising mutant sequences."

esm_prior_summaries = load_summaries(discussion_paths=[project_selection_dir / "merged.json"])
print(f"Number of prior summaries: {len(esm_prior_summaries)}")

In [ ]:
# Run ESM - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=DATA_SCIENTIST,
            agenda=esm_agenda,
            agenda_rules=coding_rules,
            save_dir=esm_dir,
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
            num_critiques=num_critiques,
            summaries=esm_prior_summaries,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

In [ ]:
# Run ESM - merge
esm_summaries = load_summaries(discussion_paths=list(esm_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(esm_summaries)}")

esm_merge_prompt = create_merge_prompt(agenda=esm_agenda)

run_individual_meeting(
    team_member=DATA_SCIENTIST,
    agenda=esm_merge_prompt,
    save_dir=esm_dir,
    save_name="merged",
    summaries=esm_summaries,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

## Improve ESM

In [ ]:
# Improve ESM - prompts
improve_esm_dir = save_dir / "improve_esm"

improve_esm_agenda = """You previously implemented a Python script that uses ESM likelihoods to identify promising mutant antibody sequence (see summary). Now, you must rewrite the script to address the following issues.

1. The code `from esm import pretrained; pretrained.Alphabet` causes an error for fair-esm version 2.0.0. Please fix all instances of `pretrained.Alphabet` by instead using the import `from esm import Alphabet`.
2. The code `model.alphabet.all_toks` causes an error for fair-esm version 2.0.0. Please fix this by instead directly using the loaded alphabet and its `alphabet.all_toks`.
3. The code does not verify that proposed high likelihood amino acids are in fact mutations (i.e., different from the input wildtype sequence). Please add a check to ensure that the proposed amino acids are different from the wildtype sequence.
4. Please check that the proposed mutations are standard amino acids (i.e., not 'X' or other non-standard amino acids).
5. The code computes mutant amino acid likelihoods but does not check whether the mutant amino acid has a higher likelihood than the wildtype amino acid. Please modify the code to only select mutants with higher likelihoods than the wildtype.
6. Please modify the code to save the top N mutant sequences with the highest ratio between the mutant likelihood and the wildtype likelihood, where N is a user-defined parameter."""

improve_esm_prior_summaries = load_summaries(discussion_paths=[project_selection_dir / "merged.json", esm_dir / "merged.json"])
print(f"Number of prior summaries: {len(improve_esm_prior_summaries)}")

In [ ]:
# Improve ESM - discussion
run_individual_meeting(
    team_member=DATA_SCIENTIST,
    agenda=improve_esm_agenda,
    agenda_rules=coding_rules,
    save_dir=improve_esm_dir,
    save_name="discussion",
    temperature=CONSISTENT_TEMPERATURE,
    num_critiques=num_critiques,
    summaries=improve_esm_prior_summaries,
    model=model,
)

## Run RosettaAntibody

In [ ]:
# Run RosettaAntibody - prompts
rosetta_dir = save_dir / "rosetta"

rosetta_agenda = f"{background_prompt} Your team previously decided on an antibody design approach using ESM and RosettaAntibody to modify existing antibodies. Now, you must implement the RosettaAntibody portion of the approach. Please write a complete Python script that takes a list of antibody sequences and an antigen structure as input and uses RosettaAntibody to rank the antibody sequences based on their binding affinity and structural stability. If needed, write a complete Rosetta XML protocol file that specifies the antibody design process."

rosetta_prior_summaries = load_summaries(discussion_paths=[project_selection_dir / "merged.json"])
print(f"Number of prior summaries: {len(rosetta_prior_summaries)}")

In [11]:
# Run RosettaAntibody - discussion
with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=COMPUTATIONAL_BIOLOGIST,
            agenda=rosetta_agenda,
            agenda_rules=coding_rules,
            save_dir=rosetta_dir,
            save_name=f"discussion_{iteration_num + 1}",
            temperature=CREATIVE_TEMPERATURE,
            num_critiques=num_critiques,
            summaries=rosetta_prior_summaries,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

Critiques (+ Final Round):   0%|          | 0/4 [00:00<?, ?it/s]

Critiques (+ Final Round):   0%|          | 0/4 [00:00<?, ?it/s]


Critiques (+ Final Round):   0%|          | 0/4 [00:00<?, ?it/s]



Critiques (+ Final Round):   0%|          | 0/4 [00:00<?, ?it/s]




Agents:   0%|          | 0/2 [00:00<?, ?it/s]





Agents:   0%|          | 0/2 [00:00<?, ?it/s]






Agents:   0%|          | 0/2 [00:00<?, ?it/s]







Agents:   0%|          | 0/2 [00:00<?, ?it/s]








Agents:   0%|          | 0/2 [00:00<?, ?it/s]





Agents:  50%|█████     | 1/2 [00:14<00:14, 14.06s/it]








Agents:  50%|█████     | 1/2 [00:14<00:14, 14.84s/it]







Agents:  50%|█████     | 1/2 [00:15<00:15, 15.04s/it]




Agents:  50%|█████     | 1/2 [00:22<00:22, 22.27s/it]





Agents: 100%|██████████| 2/2 [00:24<00:00, 12.17s/it]





                                                     

Critiques (+ Final Round):  25%|██▌       | 1/4 [00:24<01:14, 24.93s/it]





Agents:   0%|          | 0/2 [00:00<?

Input token count: 37,012
Output token count: 8,702
Max token length: 10,850
Cost: $0.32
Time: 2:12
Input token count: 37,677
Output token count: 9,180
Max token length: 11,328
Cost: $0.33
Time: 2:12










Agents: 100%|██████████| 1/1 [00:36<00:00, 36.53s/it]







                                                     


Critiques (+ Final Round): 100%|██████████| 4/4 [02:16<00:00, 35.33s/it]


                                                                        

Input token count: 36,272
Output token count: 8,722
Max token length: 10,870
Cost: $0.31
Time: 2:16











Agents: 100%|██████████| 1/1 [00:22<00:00, 22.55s/it]








                                                     



Critiques (+ Final Round): 100%|██████████| 4/4 [02:28<00:00, 36.10s/it]



                                                                        

Input token count: 41,197
Output token count: 9,361
Max token length: 11,509
Cost: $0.35
Time: 2:28









Agents: 100%|██████████| 1/1 [00:33<00:00, 33.02s/it]






                                                     
Critiques (+ Final Round): 100%|██████████| 4/4 [02:39<00:00, 39.16s/it]
                                                                        

Input token count: 39,421
Output token count: 9,402
Max token length: 11,550
Cost: $0.34
Time: 2:39


In [12]:
# Run RosettaAntibody - merge
rosetta_summaries = load_summaries(discussion_paths=list(rosetta_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(rosetta_summaries)}")

rosetta_merge_prompt = create_merge_prompt(agenda=rosetta_agenda)

run_individual_meeting(
    team_member=COMPUTATIONAL_BIOLOGIST,
    agenda=rosetta_merge_prompt,
    save_dir=rosetta_dir,
    save_name="merged",
    summaries=rosetta_summaries,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

Number of summaries: 5


Agents: 100%|██████████| 1/1 [00:24<00:00, 24.08s/it]
                                                                        

Input token count: 9,715
Output token count: 1,761
Max token length: 11,476
Cost: $0.07
Time: 0:24
